In [1]:
import sys
import time
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2

In [2]:
from seqmodel.bunch import Bunch
from seqmodel import model
from seqmodel import data

In [4]:
dec_vocab = data.Vocabulary.from_vocab_file('../data/common_wordnet_defs/lemma_senses/dec_vocab.txt')
enc_vocab = data.Vocabulary.from_vocab_file('../data/common_wordnet_defs/lemma_senses/enc_vocab.txt')
char_vocab = data.Vocabulary.from_vocab_file('../data/common_wordnet_defs/lemma_senses/char_vocab.txt')
valid_iter = data.Word2DefIterator(enc_vocab, dec_vocab, char_vocab)
valid_iter.initialize('../data/common_wordnet_defs/lemma_senses/valid.txt',
                      feature_source='../data/common_wordnet_defs/lemma_senses/valid_features.txt')
valid_iter.init_batch(4)

In [7]:
tf.reset_default_graph()
model_opt = model.definition_model.DefinitionModel.default_opt()
model_opt.embedding.decoder_dim = 10
model_opt.embedding.decoder_vocab_size = dec_vocab.vocab_size
model_opt.embedding.encoder_dim = 10
model_opt.embedding.encoder_vocab_size = enc_vocab.vocab_size
model_opt.decoder.rnn_opt.logit.out_vocab_size = dec_vocab.vocab_size
model_opt.decoder.rnn_opt.rnn_cell.cell_opt.num_units = 10
model_opt.encoder.rnn_opt.rnn_cell.cell_opt.num_units = 10
model_opt.decoder.share.encoder_rnn_params = True
model_opt.decoder.share.encoder_embedding = False
model_opt.decoder.share.logit_weight_tying = True
model_fn = model.definition_model.DefinitionModel(model_opt)
dm = model_fn(is_training=True)
optimizer = tf.train.AdamOptimizer()
g_v_pairs =  optimizer.compute_gradients(dm.training_loss, tf.trainable_variables())
train_op = optimizer.apply_gradients(g_v_pairs)
for v in tf.trainable_variables():
    print("{}, {}".format(v.name, v.get_shape()))

model/encoder_embedding:0, (31267, 10)
model/decoder_embedding:0, (27665, 10)
model/char_embedding:0, (28, 28)
model/shared_rnn/rnn/basic_lstm_cell/weights:0, (20, 40)
model/shared_rnn/rnn/basic_lstm_cell/biases:0, (40,)
model/char_cnn/filter_2:0, (1, 2, 28, 10)
model/char_cnn/filter_3:0, (1, 3, 28, 30)
model/char_cnn/filter_4:0, (1, 4, 28, 40)
model/char_cnn/filter_5:0, (1, 5, 28, 40)
model/char_cnn/filter_6:0, (1, 6, 28, 40)
model/decoder_rnn/gate_w:0, (190, 20)
model/decoder_rnn/gate_b:0, (20,)
model/decoder_rnn/logit_b:0, (27665,)


In [8]:
sess_config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())
batch = valid_iter.next_batch()
state = None

In [10]:
prediction, state, info = dm.predict(sess, batch.features)

In [11]:
eval_loss, state, info = dm.evaluate(sess, batch)

In [36]:
eval_loss, train_loss, state, info = dm.train(sess, batch, train_op)

In [37]:
eval_loss

10.156315

In [38]:
train_loss

873.44312